In [1]:

import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model, sample_gapsplit, plot_grid_95_reactions
from load_modify_sample_utils import sample_dingo, sample_optgp, sampling_statistics
from correlations_utils import correlated_reactions, plot_correlation_matrix, split_forward_reverse, compute_copula, copula_tail_dependence
from correlations_utils import find_reactants_products, sharing_metabolites, sharing_metabolites_square_matrix


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)

Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

In [3]:
ec_cobra_model_condition_1, ec_dingo_model_condition_1 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=100)
ec_cobra_model_condition_2, ec_dingo_model_condition_2 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


samples_dingo_condition_1 = sample_dingo(ec_dingo_model_condition_1, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_1.shape)

samples_dingo_condition_2 = sample_dingo(ec_dingo_model_condition_2, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_2.shape)



Read LP format model from file /tmp/tmpegbd36s7.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


Read LP format model from file /tmp/tmpil2wvuxt.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
phase 1: number of correlated samples = 500, effective sample size = 6, ratio of the maximum singilar value over the minimum singular value = 1357.28
phase 2: number of correlated samples = 500, effective sample size = 3, ratio of the maximum singilar value over the minimum singular value = 521.402
phase 3: number of correlated samples = 500, effective sample size = 4, ratio of the maximum singilar value over the minimum singular value = 306.097
phase 4: number of correlated samples = 500, effective sample size = 145, ratio of the maximum singilar value over the minimum singular value = 11.5136
phase 5: number of correlated samples = 500, effective sample size = 147, ratio of the maximum singilar value over the minimum singular value = 3.1310

[5]maximum marginal PSRF: 1.10499


phase 1: number of correlated samples = 500, effective sample size = 29, ratio of the maximum singilar value over the minimum singular value = 812.991
phase 2: number of correlated samples = 500, effective sample size = 94, ratio of the maximum singilar value over the minimum singular value = 2.31889
phase 3: number of correlated samples = 2400, effective sample size = 912
phase 4: number of correlated samples = 2400, effective sample size = 868
phase 5: number of correlated samples = 700, effective sample size = 243
[5]total ess 2146: number of correlated samples = 6500


(95, 6500)


[5]maximum marginal PSRF: 1.00555


In [5]:

extended_steady_states, extended_reactions = split_forward_reverse(samples_dingo_condition_2, reactions=ec_dingo_reactions)


reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions = find_reactants_products(ec_cobra_model, extended_reactions)
print(len(reversibility_list_all_reactions), len(reactants_list_all_reactions))


boolean_sharing_metabolites_matrix = sharing_metabolites_square_matrix(extended_reactions, reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions)


print(boolean_sharing_metabolites_matrix)
print(boolean_sharing_metabolites_matrix.shape)



127 127
[[ True False  True ... False False False]
 [False  True False ... False False False]
 [ True False  True ... False False False]
 ...
 [False False False ...  True False False]
 [False False False ... False  True False]
 [False False False ... False False  True]]
(127, 127)


In [ ]:


(linear_correlation_matrix, 
non_linear_correlation_matrix, 
mixed_correlation_matrix, 
correlations_dictionary) = correlated_reactions(
        steady_states = extended_steady_states,
        boolean_sharing_metabolites_matrix=boolean_sharing_metabolites_matrix,
        reactions=extended_reactions, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 4, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True
)




/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Completed the process of 1 from 7148 copulas
Completed the process of 2 from 7148 copulas
Completed the process of 3 from 7148 copulas
Completed the process of 4 from 7148 copulas
Completed the process of 5 from 7148 copulas
Completed the process of 6 from 7148 copulas
Completed the process of 7 from 7148 copulas
Completed the process of 8 from 7148 copulas
Completed the process of 9 from 7148 copulas
Completed the process of 10 from 7148 copulas
Completed the process of 11 from 7148 copulas
Completed the process of 12 from 7148 copulas
Completed the process of 13 from 7148 copulas
Completed the process of 14 from 7148 copulas
Completed the process of 15 from 7148 copulas
Completed the process of 16 from 7148 copulas
Completed the process of 17 from 7148 copulas
Completed the process of 18 from 7148 copulas
Completed the process of 19 from 7148 copulas
Completed the process of 20 from 7148 copulas
Completed the process of 21 from 7148 copulas
Completed the process of 22 from 7148 copul

In [11]:

filtered_positive = {
    pair: metrics
    for pair, metrics in correlations_dictionary.items()
    if metrics['jensenshannon'] > 0.15 and metrics['indicator'] > 1.5
}

print(filtered_positive)
print(filtered_positive.keys())


filtered_negative = {
    pair: metrics
    for pair, metrics in correlations_dictionary.items()
    if metrics['jensenshannon'] < -0.15 and metrics['indicator'] < 1.5

}

print(filtered_negative)
print(filtered_negative.keys())


{'PGI~PFK': {'pearson': 0, 'jensenshannon': 0.17884538027499228, 'indicator': 1.6438538181167093, 'classification': 'positive_upper_lower_tail'}, 'PGK_rev~PFK': {'pearson': 0, 'jensenshannon': 0.1813468395579364, 'indicator': 1.7066848539571813, 'classification': 'positive_upper_lower_tail'}, 'PGM_rev~PFK': {'pearson': 0, 'jensenshannon': 0.1815771714285495, 'indicator': 1.7064032669638018, 'classification': 'positive_upper_lower_tail'}, 'RPE_rev~PFK': {'pearson': 0, 'jensenshannon': 0.17151790699259073, 'indicator': 1.6130091960870867, 'classification': 'positive_upper_lower_tail'}, 'RPI~PFK': {'pearson': 0, 'jensenshannon': 0.1724561896962446, 'indicator': 1.6133069805288662, 'classification': 'positive_upper_lower_tail'}, 'ENO~PFK': {'pearson': 0, 'jensenshannon': 0.1815771714285495, 'indicator': 1.7064032669638018, 'classification': 'positive_upper_lower_tail'}, 'TALA_rev~PFK': {'pearson': 0, 'jensenshannon': 0.17187631131876627, 'indicator': 1.6185770727353443, 'classification': '

In [13]:

cobra_reactions_str = [str(reaction.id) for reaction in ec_cobra_model.reactions]

plot_correlation_matrix(linear_correlation_matrix, extended_reactions)
plot_correlation_matrix(non_linear_correlation_matrix, extended_reactions)
plot_correlation_matrix(mixed_correlation_matrix, extended_reactions)


In [15]:
from graphs_utils import construct_graph, plot_graph


G, pos = construct_graph(linear_correlation_matrix, 
                         non_linear_correlation_matrix,
                         reactions=extended_reactions,
                         remove_unconnected_nodes=True,
                         correction=False)

plot_graph(G, pos)



G, pos = construct_graph(linear_correlation_matrix,
                         non_linear_correlation_matrix=None,
                         reactions=extended_reactions,
                         remove_unconnected_nodes=True,
                         correction=False)

plot_graph(G, pos)


In [ ]:

(linear_correlation_matrix_no_metabolites, 
non_linear_correlation_matrix_no_metabolites, 
mixed_correlation_matrix_no_metabolites, 
correlations_dictionary_no_metabolites) = correlated_reactions(
        steady_states = extended_steady_states,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.1,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 4, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)


Completed the process of 1 from 7021 copulas
Completed the process of 2 from 7021 copulas
Completed the process of 3 from 7021 copulas
Completed the process of 4 from 7021 copulas
Completed the process of 5 from 7021 copulas
Completed the process of 6 from 7021 copulas
Completed the process of 7 from 7021 copulas
Completed the process of 8 from 7021 copulas
Completed the process of 9 from 7021 copulas
Completed the process of 10 from 7021 copulas
Completed the process of 11 from 7021 copulas
Completed the process of 12 from 7021 copulas
Completed the process of 13 from 7021 copulas
Completed the process of 14 from 7021 copulas
Completed the process of 15 from 7021 copulas
Completed the process of 16 from 7021 copulas
Completed the process of 17 from 7021 copulas
Completed the process of 18 from 7021 copulas
Completed the process of 19 from 7021 copulas
Completed the process of 20 from 7021 copulas
Completed the process of 21 from 7021 copulas
Completed the process of 22 from 7021 copul

In [17]:

filtered_g = {
    pair: metrics
    for pair, metrics in correlations_dictionary_no_metabolites.items()
    if metrics['jensenshannon'] > 0.1 and metrics['indicator'] > 1.2
}

print(filtered_g)
print(filtered_g.keys())


filtered_l = {
    pair: metrics
    for pair, metrics in correlations_dictionary_no_metabolites.items()
    if metrics['jensenshannon'] < -0.1 and metrics['indicator'] < 1.2

}

print(filtered_l)
print(filtered_l.keys())


{'PGI~PFK': {'pearson': 0, 'jensenshannon': 0.10810656528498305, 'indicator': 1.5967365937227758, 'classification': 'positive_upper_tail'}, 'PGK_rev~PFK': {'pearson': 0, 'jensenshannon': 0.11242950419479622, 'indicator': 1.6887096738091836, 'classification': 'positive_upper_lower_tail'}, 'PGM_rev~PFK': {'pearson': 0, 'jensenshannon': 0.11197419674800572, 'indicator': 1.6773418699738014, 'classification': 'positive_upper_lower_tail'}, 'RPE_rev~PFK': {'pearson': 0, 'jensenshannon': 0.10568556619107589, 'indicator': 1.6037441467048852, 'classification': 'positive_upper_tail'}, 'RPI~PFK': {'pearson': 0, 'jensenshannon': 0.10695426876543447, 'indicator': 1.605015670899215, 'classification': 'positive_upper_tail'}, 'ENO~PFK': {'pearson': 0, 'jensenshannon': 0.11197419674800572, 'indicator': 1.6773418699738014, 'classification': 'positive_upper_lower_tail'}, 'TALA_rev~PFK': {'pearson': 0, 'jensenshannon': 0.10571463472637252, 'indicator': 1.603124996937256, 'classification': 'positive_upper_t

In [19]:

cobra_reactions_str = [str(reaction.id) for reaction in ec_cobra_model.reactions]

plot_correlation_matrix(linear_correlation_matrix_no_metabolites, extended_reactions)
plot_correlation_matrix(non_linear_correlation_matrix_no_metabolites, extended_reactions)
plot_correlation_matrix(mixed_correlation_matrix_no_metabolites, extended_reactions)


In [14]:
import numpy as np
for index, i in enumerate(samples_dingo_condition_2):
    if np.abs(np.min(i) - np.max(i)) > 50:
        print(ec_dingo_reactions[index])

PFK
PPC
ATPM
PPCK
PPS
ADK1
ATPS4r
PYK
SUCCt2_2
CYTBD
SUCCt3
SUCDi
THD2
FBP
FORt2
FORt
FRD7
GLNS
GLUDy
GLUN
GLUSy
NADH16
NADTRHD


In [ ]:
import cobra

for i in ec_cobra_model.reactions:
    
    tmp_model = ec_cobra_model.copy()
    tmp_model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").objective_coefficient = 0
    tmp_model.reactions.get_by_id(i.id).objective_coefficient = 1
    fva = cobra.flux_analysis.flux_variability_analysis(ec_cobra_model,fraction_of_optimum=0)
    print(fva[fva.index==i.id])
    

Read LP format model from file /tmp/tmpe1cybd3n.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


     minimum  maximum
PFK      0.0   176.61
Read LP format model from file /tmp/tmp5wrva1g2.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros
     minimum  maximum
PFL      0.0     40.0
Read LP format model from file /tmp/tmplyv_b_5n.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros
     minimum  maximum
PGI    -50.0     10.0
Read LP format model from file /tmp/tmpvtu2hp4q.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
     minimum  maximum
PGK    -20.0      0.0
Read LP format model from file /tmp/tmp_9li36ue.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros
     minimum  maximum
PGL      0.0     60.0
Read LP format model from file /tmp/tmp4_fl2x5y.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
       minimum  maximum
ACALD    -20.0      0.0
Read LP format model from file /tmp/tmpf3rjg3fq.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
        minimum  maximum
AKGt2r    -